In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector as mdb
import ujson as js
import torch as pt
import tqdm

sns.set()

from utils.market_parser import market_smooth
from utils.sql_pullers import get_types

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
def mat_eff_calc(mat_quant, me_level):
    quant_actual = np.maximum(1, np.ceil(np.round(mat_quant * ((100 - me_level) / 100), 2))).astype(int)
    return quant_actual

In [3]:
with open('./utils/sql_calls.json') as file:
    sql = js.load(file)

mat_region_id = 10000043
sale_region_id = 10000052
#sale_region_id = 10000043

h = 14
markets = {}
prices = {}

market_start = '2019-01-01'

In [4]:
with open('./../settings/maria_login.json') as file:
    conn = mdb.connect(**js.load(file))

In [5]:
corp_bps = pd.read_sql(sql['corp']['bpos'], conn, index_col='bp_item_id')
corp_bps = corp_bps.loc[corp_bps[['bp_type_id', 'mat_eff', 'time_eff']].drop_duplicates().index]
corp_bps.index = corp_bps.index.astype(str)

In [6]:
corp_orders = pd.read_sql(sql['corp']['orders'], conn, index_col='order_id')

In [7]:
bp_data = {}
for key, script in sql['bp'].items():
    bp_data[key] = pd.read_sql(script.format(type_ids=','.join(corp_bps['bp_type_id'].astype(str))), conn)

In [8]:
invent = {
    'mats': {
        'quant': corp_bps.reset_index().merge(
            bp_data['mats'].loc[bp_data['mats']['act_id'] == 8],
            on='bp_type_id'
        )[['bp_item_id', 'bp_type_id', 'mat_eff', 'mat_type_id', 'mat_quant']].pivot(
            index='bp_item_id',
            columns='mat_type_id',
            values='mat_quant'
        ).fillna(0)
    }
}

In [9]:
markets['invent'] = market_smooth(mat_region_id, invent['mats']['quant'].columns, h, start_date=market_start)

prices[mat_region_id] = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['invent'][mat_region_id].items()]
).set_index('type_id')

invent['mats']['price'] = pd.concat([
    (prices[mat_region_id]['avg_price'] * invent['mats']['quant']).sum(axis=1),
    (prices[mat_region_id]['variance'] * (invent['mats']['quant']**2)).sum(axis=1)
], axis=1)
invent['mats']['price'].columns = prices[mat_region_id].columns

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
invent['prods'] = corp_bps.reset_index().merge(
    bp_data['prods'].loc[bp_data['prods']['act_id'] == 8],
    on='bp_type_id'
)[['bp_item_id', 'bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']]
invent['prods']['avg_price_run'] = invent['prods']['bp_item_id'].map(invent['mats']['price']['avg_price'])
invent['prods']['variance_run'] = invent['prods']['bp_item_id'].map(invent['mats']['price']['variance'])
invent['prods']['avg_price_unit'] = invent['prods']['avg_price_run'] / invent['prods'][['prod_quant', 'prod_prob']].product(axis=1)
invent['prods']['variance_unit'] = invent['prods']['variance_run'] * (invent['prods'][['prod_quant', 'prod_prob']].product(axis=1)**2)
invent['prods'].index = invent['prods']['bp_item_id'] + ':' + invent['prods']['prod_type_id'].astype(str)
invent['prods'].index.name = 'bp_item_id'
invent['prods'].drop(['bp_item_id', 'bp_type_id'], axis=1, inplace = True)

In [11]:
invent_bps = corp_bps.loc[invent['prods'].index.to_series().apply(lambda x: x.split(':')[0])]
invent_bps.index = invent['prods'].index
invent_bps['bp_type_id'] = invent['prods']['prod_type_id']
invent_bps['mat_eff'] = 2
invent_bps['time_eff'] = 4
corp_bps = corp_bps.append(invent_bps)
corp_bps.index.name = 'bp_item_id'
del invent_bps

In [12]:
for key, sql in sql['bp'].items():
    bp_data[key] = bp_data[key].append(
        pd.read_sql(sql.format(type_ids=','.join(invent['prods']['prod_type_id'].unique().astype(str))), conn),
        ignore_index=True
    ).drop_duplicates().reset_index(drop=True)

In [13]:
build = {
    'count': corp_bps.reset_index().merge(
        bp_data['mats'].loc[bp_data['mats']['act_id'] == 1],
        on='bp_type_id'
    )[['bp_item_id', 'bp_type_id', 'mat_eff', 'mat_type_id', 'mat_quant']]
}
build['count']['mat_actual'] = np.maximum(1, np.ceil(np.round(build['count']['mat_quant'] * ((100 - build['count']['mat_eff']) / 100), 2)))
build['count'] = build['count'].pivot(index='bp_item_id', columns='mat_type_id', values='mat_actual').fillna(0)

In [14]:
sub_prods = bp_data['prods'].loc[
    (bp_data['prods']['prod_type_id'].isin(build['count'].columns)) & (bp_data['prods']['act_id'] == 1)
][['bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']].reset_index(drop=True)
sub_prods = sub_prods.merge(bp_data['mats'].loc[bp_data['mats']['act_id'] == 1, ['bp_type_id', 'mat_type_id', 'mat_quant']], on='bp_type_id')
sub_prods = corp_bps.reset_index()[['bp_item_id', 'bp_type_id', 'mat_eff']].merge(sub_prods, on='bp_type_id')
sub_prods['mat_actual'] = mat_eff_calc(sub_prods['mat_quant'], sub_prods['mat_eff'])
sub_prods['mat_unit'] = sub_prods['mat_actual'] / (sub_prods['prod_quant'] * sub_prods['prod_prob'])
sub_prods['bp_item_id.prod_type_id'] = sub_prods['bp_item_id'] + '.' + sub_prods['prod_type_id'].astype(str)
sub_prods = sub_prods.pivot(index='bp_item_id.prod_type_id', columns='mat_type_id', values='mat_unit').fillna(0)
sub_prods.index = pd.Series([int(col.split('.')[1]) for col in sub_prods.index], name='prod_type_id')

active = True
while active:
    build_mod = build['count'][sub_prods.index] @ sub_prods
    build['count'][sub_prods.index] = 0
    build['count'].loc[build_mod.index, build_mod.columns] += build_mod
    build['count'] = build['count'].loc[:, build['count'].sum(axis=0) > 0]
    
    sub_prods = bp_data['prods'].loc[
        (bp_data['prods']['prod_type_id'].isin(build['count'].columns)) & (bp_data['prods']['act_id'] == 1)
    ][['bp_type_id', 'prod_type_id', 'prod_quant', 'prod_prob']].reset_index(drop=True)
    sub_prods = sub_prods.merge(bp_data['mats'].loc[bp_data['mats']['act_id'] == 1, ['bp_type_id', 'mat_type_id', 'mat_quant']], on='bp_type_id')
    sub_prods = corp_bps.reset_index()[['bp_item_id', 'bp_type_id', 'mat_eff']].merge(sub_prods, on='bp_type_id')
    sub_prods['mat_actual'] = mat_eff_calc(sub_prods['mat_quant'], sub_prods['mat_eff'])
    sub_prods['mat_unit'] = sub_prods['mat_actual'] / (sub_prods['prod_quant'] * sub_prods['prod_prob'])
    sub_prods['bp_item_id.prod_type_id'] = sub_prods['bp_item_id'] + '.' + sub_prods['prod_type_id'].astype(str)
    sub_prods = sub_prods.pivot(index='bp_item_id.prod_type_id', columns='mat_type_id', values='mat_unit').fillna(0)
    sub_prods.index = pd.Series([int(col.split('.')[1]) for col in sub_prods.index], name='prod_type_id')
    
    if len(sub_prods) == 0: active = False

In [15]:
markets['build'] = market_smooth(mat_region_id, build['count'].columns, h, start_date=market_start)

prices[mat_region_id] = prices[mat_region_id].append(pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['build'][mat_region_id].items()]
).set_index('type_id')).drop_duplicates().sort_index()

In [16]:
build['price'] = pd.concat([
    (build['count'] * prices[mat_region_id]['avg_price']).sum(axis=1),
    ((build['count']**2) * prices[mat_region_id]['variance']).sum(axis=1)
], axis=1)
build['price'].columns = ['%s_run' % col for col in prices[mat_region_id].columns]

In [17]:
build['prods'] = corp_bps.reset_index().merge(
    bp_data['prods'].loc[bp_data['prods']['act_id'] == 1],
    on='bp_type_id'
)[['bp_item_id', 'prod_type_id', 'prod_quant', 'prod_prob']]
build['prods']['avg_price_run'] = build['prods']['bp_item_id'].map(build['price']['avg_price_run'])
build['prods']['variance_run'] = build['prods']['bp_item_id'].map(build['price']['variance_run'])
build['prods'].set_index('bp_item_id', inplace=True)

In [18]:
build['cost'] = build['prods'][['prod_type_id', 'prod_quant', 'prod_prob', 'avg_price_run', 'variance_run']].join(
    invent['prods'][['avg_price_unit', 'variance_unit']]
).fillna(0)
build['cost'].rename(columns={'avg_price_unit':'avg_price_invent', 'variance_unit':'variance_invent'}, inplace=True)

In [19]:
build['cost']['avg_price_unit'] = (
    build['cost']['avg_price_run'] + build['cost']['avg_price_invent']
) / (
    build['cost']['prod_quant'] * build['cost']['prod_prob']
)
build['cost']['variance_unit'] = (
    build['cost']['variance_run'] + build['cost']['variance_invent']
) / (
    (build['cost']['prod_quant'] * build['cost']['prod_prob'])**2
)

In [20]:
markets['sale'] = market_smooth(sale_region_id, build['cost']['prod_type_id'].unique(), h, start_date=market_start)

prices[sale_region_id] = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_price': vals[h]['level'].iloc[-1]['avg_price'],
        'variance': vals[h]['var'].iloc[-1]['avg_price']
    } for type_id, vals in markets['sale'][sale_region_id].items()]
).set_index('type_id')

In [21]:
build['cost']['avg_price_sale'] = build['cost']['prod_type_id'].map(prices[sale_region_id]['avg_price'])
build['cost']['variance_sale'] = build['cost']['prod_type_id'].map(prices[sale_region_id]['variance'])
build['cost']['avg_profit'] = build['cost']['avg_price_sale'] - build['cost']['avg_price_unit']
build['cost']['profit_ratio'] = build['cost']['avg_profit'] / build['cost']['avg_price_unit']
build['cost']['variance_profit'] = build['cost']['variance_sale'] + build['cost']['variance_unit']
build['cost']['stdev_profit'] = np.sqrt(build['cost']['variance_profit'])
build['cost']['z_profit'] = build['cost']['avg_profit'] / (np.sqrt(build['cost']['variance_sale']) + np.sqrt(build['cost']['variance_unit']))

In [22]:
types = get_types(build['cost']['prod_type_id'])
build['cost']['prod_type_name'] = build['cost']['prod_type_id'].map(types['type_name'])
build['cost']['prod_group_name'] = build['cost']['prod_type_id'].map(types['group_name'])
build['cost']['prod_cat_name'] = build['cost']['prod_type_id'].map(types['category_name'])

In [23]:
volume = pd.DataFrame(
    [{
        'type_id': type_id,
        'avg_volume': vals[h]['level'].iloc[-1]['volume'],
        'variance': vals[h]['var'].iloc[-1]['volume']
    } for type_id, vals in markets['sale'][sale_region_id].items()]
).set_index('type_id')

build['cost']['volume'] = build['cost']['prod_type_id'].map(volume['avg_volume'])
build['cost']['vol_var'] = build['cost']['prod_type_id'].map(volume['variance'])
build['cost']['avg_value'] = build['cost']['avg_profit'] * build['cost']['volume']
build['cost']['var_value'] = (
    build['cost']['variance_profit'] * build['cost']['vol_var']
) + (
    build['cost']['variance_profit'] * (build['cost']['volume']**2)
) + (
    (build['cost']['avg_profit']**2) * build['cost']['vol_var']
)
build['cost']['stdev_value'] = np.sqrt(build['cost']['var_value'])

In [24]:
build['cost']['active_order'] = build['cost']['prod_type_id'].isin(corp_orders['type_id'])

In [25]:
build['cost'].loc[
    ~build['cost']['active_order'],
    [
        'prod_cat_name', 'prod_group_name', 'prod_type_name', 'avg_price_unit',
        'avg_price_sale', 'avg_profit', 'profit_ratio', 'stdev_profit', 'volume',
        'avg_value'
    ]
].sort_values('avg_value', ascending=False).iloc[:50]

,prod_cat_name,prod_group_name,prod_type_name,avg_price_unit,avg_price_sale,avg_profit,profit_ratio,stdev_profit,volume,avg_value
bp_item_id,,,,,,,,,,
1021711311640,Ship,Mining Barge,Procurer,"13,741,070.22","18,286,332.83","4,545,262.61",0.33,"2,035,894.88",1.34,"6,107,542.67"
1000184327707,Module,Strip Miner,Strip Miner I,"892,046.75","1,225,481.60","333,434.85",0.37,"459,767.28",9.21,"3,070,352.61"
1022148045264:31725,Module,Rig Shield,Medium Anti-EM Screen Reinforcer II,"2,722,008.80","14,519,879.78","11,797,870.97",4.33,"629,582.59",0.17,"1,950,416.88"
567672053:2176,Drone,Combat Drone,Infiltrator II,"734,328.55","1,303,218.64","568,890.09",0.77,"208,011.38",2.96,"1,686,566.11"
1236757874:32048,Module,Rig Drones,Medium Drone Mining Augmentor II,"21,526,797.13","33,189,907.28","11,663,110.15",0.54,"764,714.74",0.13,"1,462,659.88"
1544516305:19740,Module,Missile Launcher Cruise,Cruise Missile Launcher II,"2,489,300.03","4,341,951.64","1,852,651.62",0.74,"275,695.29",0.73,"1,347,624.19"
516893894,Charge,Auto-Targeting Cruise Missile,Nova Auto-Targeting Cruise Missile I,114.85,244.83,129.98,1.13,98.13,"10,333.42","1,343,172.55"
962734487:12104,Module,Cloaking Device,Improved Cloaking Device II,"2,688,918.76","5,915,842.83","3,226,924.07",1.20,"1,545,244.46",0.26,"832,137.47"
732135011,Ship,Combat Battlecruiser,Prophecy,"36,210,139.05","38,910,382.12","2,700,243.08",0.07,"12,983,332.73",0.28,"765,117.09"


In [26]:
with open('./../settings/maria_login.json') as file:
    conn = mdb.connect(**js.load(file))

with open('./utils/sql.json') as file:
    undercuts = pd.read_sql(js.load(file)['undercuts'], conn, index_col = 'order_id')
undercuts['market_volume_sum'] = undercuts['market_volume_sum'].astype(int)
types = types.append(get_types([type_id for type_id in undercuts['type_id'].unique() if type_id not in types.index])).sort_index()
undercuts.insert(
    undercuts.columns.get_loc('type_id') + 1,
    'type_name',
    undercuts['type_id'].map(types['type_name'])
)

conn.close()

thresh = 0.1

mask = undercuts['volume'] * thresh <= undercuts['market_volume_sum']

print(undercuts.loc[mask].shape)
undercuts.loc[mask].sort_values('type_name', ascending=True).iloc[:50]

(4, 9)


,type_id,type_name,region_id,price,volume,market_order_count,market_price_min,market_volume_sum,market_volume_max
order_id,,,,,,,,,
5516950854,643,Armageddon,10000052,"160,000,000.00",1,2,"150,000,100.09",2,1
5512405324,4247,Helium Fuel Block,10000052,"24,000.00",4600,1,"19,000.00",18718,18718
5496665892,204,Inferno Cruise Missile,10000052,165.00,98000,2,145.00,148866,144866
5511298884,4312,Oxygen Fuel Block,10000052,"25,000.00",10000,1,"22,000.00",8000,8000


In [27]:
print(undercuts.loc[~mask].shape)
undercuts.loc[~mask].sort_values('type_name', ascending=True).iloc[:50]

(72, 9)


,type_id,type_name,region_id,price,volume,market_order_count,market_price_min,market_volume_sum,market_volume_max
order_id,,,,,,,,,
5483836089,12344,200mm Railgun I,10000052,"95,000.00",100,1,"86,000.00",4,4
5504398645,493,720mm Howitzer Artillery I,10000052,"400,000.00",100,1,"360,000.00",1,1
5483836091,561,75mm Gatling Rail I,10000052,"17,500.00",84,1,"2,000.00",3,3
5482099146,42832,Armor Energizing Charge,10000052,600.00,6900,1,540.00,470,470
5482099152,3554,Cap Booster 100,10000052,750.00,4652,1,680.00,50,50
5482099153,11285,Cap Booster 200,10000052,"21,750.00",2263,1,"19,000.00",82,82
5482099154,11287,Cap Booster 400,10000052,"7,000.00",4613,1,"6,250.00",20,20
5479289548,186,Carbonized Lead M,10000052,17.50,95000,1,13.00,2000,2000
5479289549,178,Carbonized Lead S,10000052,18.75,65000,1,17.00,2800,2800
